In [9]:
from pybtex.database import parse_file
import json
from pprint import pprint
import os

In [2]:
# changes names format to a single string (ex: "Robert Downey Jr.")

def name_filtering(data,test):
    for j in test :
        dataset = []
        if 'author' in test[j].persons:
            str = 'author'
        elif 'editor' in test[j].persons: 
            str = 'editor'
        else: 
            continue
        for i in test[j].persons[str]:
            first = ""
            middle = ""
            last = ""
            line = ""
            all = ""
            if i.first_names:
                all += i.first_names[0]
            if i.middle_names:
                all += " " + i.middle_names[0]
            if i.last_names:
                all += " " + i.last_names[0]
            if i.lineage_names:
                all += " " + i.lineage_names[0]
            dataset.append(all)
        data[j] = {'names': dataset, 'year': test[j].fields['year'], 'title': test[j].fields['title']}
    return data

In [3]:
# creates links and nodes 

def setup(data,ref_list,to_import):
    jfile = {}
    jfile["nodes"] = []
    jfile["links"] = []
    for article in data :
        to_do = False
        for k in data[article]['names']:
            if k in ref_list:
                to_do = True
                break
        if to_do:
            for i in data[article]['names']:
                node = {}
                node["id"] = i
                node["lastColaboration"] = "0"
                if i in ref_list:
                    node["group"] = find_team(i,to_import)
                else:
                    node["group"] = "no team"
                new_node = True
                for _node in jfile["nodes"]:
                    if _node["id"] == i:
                        new_node = False
                if new_node:
                    jfile["nodes"].append(node)
                for j in data[article]['names'] :
                    if (i != j) :
                        new = True
                        for k in jfile["links"]:
                            if (k["source"] == j) & (k["target"] == i) :
                                new = False
                                break
                            elif (k["source"] == i) & (k["target"] == j) :
                                if data[article]['title'] not in k["titles"]:
                                    k['titles'].append(data[article]['title'])
                                    k["nbLinks"] += 1
                                
                                link["lastColaboration"] = max(link["lastColaboration"],data[article]['year'])
                                new = False
                                break
                        if new:
                            link = {}
                            link["source"] = i
                            link["target"] = j
                            link["nbLinks"] = 1
                            link["titles"] = [data[article]['title']]
                            link["lastColaboration"] = data[article]['year']
                            jfile["links"].append(link)
                                                        
                for j in range(len(jfile["nodes"])):
                    if jfile["nodes"][j]["id"] == i:
                        jfile["nodes"][j]["lastColaboration"] = max(jfile["nodes"][j]["lastColaboration"],data[article]['year'])
                        break
                    
    return jfile


In [4]:
def clean_data(data):
    tab = []
    tab.append(['{\\\'{e}}','é'])
    tab.append(['{\\\'{E}}','É'])
    tab.append(['{-}','-'])
    tab.append(['{\\^{o}}','ô'])
    tab.append(['{\\"{e}}','ë'])
    tab.append(['{\\"{u}}','ü'])
    tab.append(['{\\`{e}}','è'])
    tab.append(['{\\`{o}}','ò'])
    tab.append(['{\\"{a}}','ä'])
    tab.append(['{\\ae}','æ'])
    tab.append(['{\\ss}','ß'])
    tab.append(['{\\"{o}}','ö'])
    tab.append(['{\\"{\\i}}','ï'])
    tab.append(['{\\\'{a}}','à'])
    tab.append(['{\\^{\\i}}','î'])
    tab.append(['{\\\'{\\i}}','í'])
    tab.append(['{\\~{n}}','ñ'])
    tab.append(['{\\c{c}}','ç'])
    tab.append(['{\\c{C}}','Ç'])
    tab.append(['{\oe}','œ'])
    tab.append(['{\\`{a}}','à'])
    tab.append(['{\\(\\mathscr{l}\\)}','ℓ'])
    tab.append(['\\emph{delta}','δ'])
    tab.append(['{\\^{a}}','â'])
    tab.append(['{\\&}','&'])
    tab.append(['{\\\'{o}}','ó'])
    tab.append(['{\\(\\mu\\)}','μ'])
    tab.append(['{\\~{a}}','ã'])
    tab.append(['{\\"{O}}','Ö'])
    tab.append(['{\\^{e}}','ê'])
    tab.append(['\\{é\\}','é'])
    
    
    for article in data :
        for pair in tab:
            for i in range(len(data[article]['names'])):
                data[article]['names'][i] = data[article]['names'][i].replace(pair[0],pair[1])
            data[article]['title'] = data[article]['title'].replace(pair[0],pair[1])
        

In [5]:
#save results in json file

def save(jfile):
    with open('data.json', 'w') as f:
        json.dump(jfile, f)

    pprint(jfile)

In [6]:
def retrieve_files():
    to_import = []
    for x in os.walk(os.getcwd() + "\\bibtex_data"):
        if x[2]:
            team_name = x[0].replace(os.getcwd() + "\\bibtex_data\\", '')
            data = {'team': team_name, 'persons': []}
            for i in x[2]:
                i = i.replace('.bib','')
                data['persons'].append(i.split('_'))
            to_import.append(data)

    ref_list = []
    for i in range(len(to_import)):
        for j in to_import[i]['persons']:
            ref_list += [j[1] + ' ' + j[0]]
    
    return to_import,ref_list

In [7]:
def find_team(name,to_import):
    usable_name = list(reversed(name.split(" ")))
    if len(usable_name) > 2:
        usable_name = [" ".join(reversed(usable_name[0:len(usable_name)-1])),usable_name[len(usable_name)-1]]
    for team in to_import:
        if usable_name in team["persons"]:
            return team["team"]
    return "unknown_team"

In [10]:
filtered_data = {}

to_import,ref_list = retrieve_files()

for team_names in to_import:
    for people in team_names['persons']:
        data = parse_file('bibtex_data/' + team_names['team'] + '/'+ people[0] + '_' + people[1] + '.bib').entries
        filtered_data = name_filtering(filtered_data,data)
        clean_data(filtered_data)
       
jfile = setup(filtered_data,ref_list,to_import)
save(jfile)

{'links': [{'lastColaboration': '2018',
            'nbLinks': 1,
            'source': 'Reza Babajanivalashedi',
            'target': 'Armand Baboli',
            'titles': ['A Predictive Approach to Define the Best Forecasting '
                       'Method for Spare Parts: {A} Case Study in Business '
                       "Aircrafts' Industry"]},
           {'lastColaboration': '2018',
            'nbLinks': 1,
            'source': 'Reza Babajanivalashedi',
            'target': 'Muhammad Kashif Shahzad',
            'titles': ['A Predictive Approach to Define the Best Forecasting '
                       'Method for Spare Parts: {A} Case Study in Business '
                       "Aircrafts' Industry"]},
           {'lastColaboration': '2018',
            'nbLinks': 1,
            'source': 'Reza Babajanivalashedi',
            'target': 'Romy Tonadre',
            'titles': ['A Predictive Approach to Define the Best Forecasting '
                       'Method for Spare Part

            'titles': ['A rewarding-punishing coordination mechanism based on '
                       'Trust in a divergent supply chain']},
           {'lastColaboration': '2013',
            'nbLinks': 1,
            'source': 'Mohammad Modarres',
            'target': 'Mohammad R. Jokar',
            'titles': ['A rewarding-punishing coordination mechanism based on '
                       'Trust in a divergent supply chain']},
           {'lastColaboration': '2013',
            'nbLinks': 2,
            'source': 'Mansour Rached',
            'target': 'Zied Bahroun',
            'titles': ['Mathematical model for assignment policies and '
                       'information sharing in a supply chain',
                       'The Value of Information Sharing in a Serial Supply '
                       'Chain with Centralised and Decentralised Decision']},
           {'lastColaboration': '2013',
            'nbLinks': 2,
            'source': 'Mansour Rached',
            'target':

            'source': 'Thomas Cerqueus',
            'target': 'Damien Levin',
            'titles': ['Mysins : make your semantic INformation system']},
           {'lastColaboration': '2010',
            'nbLinks': 1,
            'source': 'Louis-Alexandre Celton',
            'target': 'Gaëtan Hervouet',
            'titles': ['Mysins : make your semantic INformation system']},
           {'lastColaboration': '2010',
            'nbLinks': 1,
            'source': 'Louis-Alexandre Celton',
            'target': 'Damien Levin',
            'titles': ['Mysins : make your semantic INformation system']},
           {'lastColaboration': '2010',
            'nbLinks': 1,
            'source': 'Louis-Alexandre Celton',
            'target': 'Philippe Lamarre',
            'titles': ['Mysins : make your semantic INformation system']},
           {'lastColaboration': '2010',
            'nbLinks': 1,
            'source': 'Louis-Alexandre Celton',
            'target': 'Sylvie Cazalens',
   

            'nbLinks': 1,
            'source': 'Wlodzimierz Drabent',
            'target': 'Artur Wilk',
            'titles': ['Descriptive Typing Rules for Xcerpt']},
           {'lastColaboration': '2013',
            'nbLinks': 4,
            'source': 'Ansem Ben Cheikh',
            'target': 'Agnès Front',
            'titles': ['E-CARe: {A} Process for Engineering Ubiquitous '
                       'Information Systems',
                       'An engineering method for context-aware and reactive '
                       'systems',
                       'Une modélisation évènementielle des {SI} ubiquitaires',
                       'Event Based Modeling for Context-Reactive Information '
                       'Systems']},
           {'lastColaboration': '2013',
            'nbLinks': 4,
            'source': 'Ansem Ben Cheikh',
            'target': 'Jean-Pierre Giraudin',
            'titles': ['E-CARe: {A} Process for Engineering Ubiquitous '
                       'Infor

            'target': 'Nicolas Lumineau',
            'titles': ['A Novel Vision for Navigation and Enrichment in '
                       'Cultural Heritage Collections']},
           {'lastColaboration': '2014',
            'nbLinks': 2,
            'source': 'Bilal Berjawi',
            'target': 'Elisabeth Chesneau',
            'titles': ['Uncertainty visualization of multi-providers '
                       'cartographic integration',
                       'Representing Uncertainty in Visual Integration']},
           {'lastColaboration': '2014',
            'nbLinks': 1,
            'source': 'Bilal Berjawi',
            'target': 'Geoffrey Seccia',
            'titles': ['Uncertainty visualization of multi-providers '
                       'cartographic integration']},
           {'lastColaboration': '2014',
            'nbLinks': 2,
            'source': 'Fabien Duchateau',
            'target': 'Elisabeth Chesneau',
            'titles': ['Uncertainty visualization of multi

...

                       "l'expression du besoin d'information"]},
           {'lastColaboration': '2000',
            'nbLinks': 1,
            'source': 'Philippe Bigeon',
            'target': 'Denis Malavieille',
            'titles': ["Intégration de l'indexation conceptuelle dans "
                       "l'expression du besoin d'information"]},
           {'lastColaboration': '2000',
            'nbLinks': 1,
            'source': 'Philippe Bigeon',
            'target': 'Diane Penel',
            'titles': ["Intégration de l'indexation conceptuelle dans "
                       "l'expression du besoin d'information"]},
           {'lastColaboration': '2000',
            'nbLinks': 1,
            'source': 'Philippe Bigeon',
            'target': 'Dominique L. Scapin',
            'titles': ["Intégration de l'indexation conceptuelle dans "
                       "l'expression du besoin d'information"]},
           {'lastColaboration': '2000',
            'nbLinks': 1,
            

           {'group': 'no team',
            'id': 'Daisuke Takayama',
            'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Shigeru Takano',
            'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Julien Cumin',
            'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Dimitris Kotzinos',
            'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Dominique Laurent',
            'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Nicolas Spyratos',
            'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Yuzuru Tanaka',
            'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Jean-Marc Lacombe',
            'lastColaboration': '2016'},
           {'group': 'no team',
            'id': 'Lhouari Nourine',
            'lastColaboration': '2016'},
        

            'lastColaboration': '2010'},
           {'group': 'no team', 'id': 'Emre Dogan', 'lastColaboration': '2018'},
           {'group': 'no team', 'id': 'Gonen Eren', 'lastColaboration': '2018'},
           {'group': 'no team',
            'id': 'Christian Wolf',
            'lastColaboration': '2018'},
           {'group': 'no team',
            'id': 'Eric Lombardi',
            'lastColaboration': '2018'},
           {'group': 'no team',
            'id': 'Yiqiang Chen',
            'lastColaboration': '2018'},
           {'group': 'IMAGINE',
            'id': 'Stefan Duffner',
            'lastColaboration': '2018'},
           {'group': 'no team',
            'id': 'Andrei Stoian',
            'lastColaboration': '2018'},
           {'group': 'no team',
            'id': 'Jean-Yves Dufour',
            'lastColaboration': '2018'},
           {'group': 'no team',
            'id': 'Lilei Zheng',
            'lastColaboration': '2018'},
           {'group': 'IMAGINE',
       

           {'group': 'no team',
            'id': 'A. El Kharraz',
            'lastColaboration': '2008'},
           {'group': 'no team',
            'id': 'Emilie Naud',
            'lastColaboration': '2007'},
           {'group': 'no team',
            'id': 'Çagatay Dikici',
            'lastColaboration': '2006'},
           {'group': 'no team',
            'id': 'Radhouane Guermazi',
            'lastColaboration': '2005'},
           {'group': 'no team', 'id': 'Phuc Ngo', 'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Isabelle Debled-Rennesson',
            'lastColaboration': '2017'},
           {'group': 'IMAGINE',
            'id': 'Bertrand Kerautret',
            'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Hayat Nasser',
            'lastColaboration': '2017'},
           {'group': 'no team',
            'id': 'Yukiko Kenmochi',
            'lastColaboration': '2017'},
           {'group': 'no team',
   

            'lastColaboration': '2004'},
           {'group': 'no team',
            'id': 'Vassilissa Lehoux',
            'lastColaboration': '2018'},
           {'group': 'no team',
            'id': 'Tetsuya Yoshida',
            'lastColaboration': '2007'},
           {'group': 'no team',
            'id': 'Kassem Kallel',
            'lastColaboration': '2007'},
           {'group': 'no team',
            'id': 'Gerardo Gutiérrez Segura',
            'lastColaboration': '2004'},
           {'group': 'no team',
            'id': 'Chamsi Abu Rana',
            'lastColaboration': '2014'},
           {'group': 'no team',
            'id': 'Fayyad Usama',
            'lastColaboration': '2014'},
           {'group': 'no team',
            'id': 'Milad Alshomary',
            'lastColaboration': '2014'},
           {'group': 'SMA',
            'id': 'Christine Gertosio',
            'lastColaboration': '2014'},
           {'group': 'no team',
            'id': 'Laurianne Charrier',
  